[View in Colaboratory](https://colab.research.google.com/github/pamsfwang/perceptual_learning/blob/master/stimuli_neural_model.ipynb)

In [0]:
import tensorflow as tf
import numpy
import os
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
#Mount Google drive on Colab
#step01
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
#step02: code for mounting google drive on Linux
!mkdir -p drive
!google-drive-ocamlfuse drive

#step03: change directory to the google drive folder for the project
os.getcwd()
!ls
os.chdir('./drive/Colab_files')
os.getcwd()
!ls

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
datalab  drive
drive			     notebooks
jfj_stimuli_param.csv	     stimuli_param_uniqueF3.csv
jfj_stimuli_param_test.csv   stimuli_param_uniqueF3_test_new.csv
jfj_stimuli_param_train.csv  stimuli_param_uniqueF3_train_new.csv
models


In [79]:
#load data
df=pd.read_csv('stimuli_param_uniqueF3_train_new.csv', sep=',',header=None)
df.shape
nsamples =df.shape[0]
print "number of samples:",nsamples

##separate data into features and labels (numpy.array)
temp = df.iloc[:,0:3]
x_data = temp.values
print "dimension of feature dataset:",x_data.shape
print(x_data[0:5,:])

temp = df.iloc[:,3]
y_data = temp.values
print "dimension of category dataset:",y_data.shape
print(y_data[0:5])

number of samples: 52
dimension of feature dataset: (52, 3)
[[1.  1.  2.2]
 [1.  1.  2.5]
 [1.  2.  5. ]
 [1.  2.  0.3]
 [1.  3.  3.1]]
dimension of category dataset: (52,)
[1 1 1 1 1]


In [0]:
#You can think of most the tensorflow variables as actually being functions that we will call when we want to get their value. 
input_ph = tf.placeholder(tf.float32, shape=[3,1]) #This will be the place the input to the network is inserted
target_ph =  tf.placeholder(tf.float32, shape=[1,1]) #This will be the place the target for the network is insertedd
W1 = tf.Variable(tf.random_uniform([10,3],-1,1)) #First layer weights
b1 = tf.Variable(tf.random_uniform([10,1],-1,1)) # " " biases
#[# nodes, # nodes], sampling from a random uniform distribution from -1 to 1 for initial weights
W2 = tf.Variable(tf.random_uniform([1,10],-1,1)) #2nd layer
b2 = tf.Variable(tf.random_uniform([1],-1,1))
h1 = tf.nn.tanh(tf.matmul(W1,input_ph)+b1)
output = tf.nn.tanh(tf.matmul(W2,h1)+b2) #This is the actual construction of the network. When we want to get the output of the network, we will tell tensorflow what to put in the input placeholder, and then we'll run this output function

loss = tf.reduce_sum(tf.square(output - target_ph)) #This is the function we're trying to optimize. The reduce_sum is not really necessary since we only have a single output, just using it to flatten the output.
optimizer = tf.train.AdamOptimizer(0.005) #This is a fancy version of momentum based gradient descent optimization.
train = optimizer.minimize(loss) #This will be how we tell the network to train on an example


In [0]:
# Launch the graph -- tell tensorflow to initialize everything.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) #first argument to sess.run is function to run, here we're running the initialize function


In [0]:
def test():
    MSE = 0.0
    for i in xrange(len(x_data)):
      MSE += sess.run(loss,feed_dict={input_ph: x_data[i].reshape([3,1]),target_ph: y_data[i].reshape([1,1])}) #test on a test data point. feed_dict is how you pass things in to the placeholders created above
      MSE /= 4
    return MSE

def accuracy():
    ac = []
    for i in xrange(len(test_y_data)):
        ac[i]=sess.run(output,feed_dict={input_ph: test_x_data[i].reshape([3,1])})
    return ac
    
    
def hidden():
    hidden = []
    for i in xrange(len(test_y_data)):
        hidden[i]=sess.run(h1,feed_dict={input_ph: test_x_data[i].reshape([3,1])})
    return hidden

In [70]:
# Fit the function
print "Pre training MSE:", test
hi = []
for step in xrange(1000):
    sess.run(train,feed_dict={input_ph: x_data[step % nsamples].reshape([3,1]),target_ph: y_data[step % nsamples].reshape([1,1])}) 
    #Run training on an example
    #hi.append(hidden())
    if step % 100 == 0:
	print "On step %i, test MSE %f" %(step,test())

print "Post training MSE:", test()

print "Final weights:"
print(sess.run(W1),sess.run(b1),sess.run(W2),sess.run(b2))

Pre training MSE: <function test at 0x7f46ff8ad5f0>
On step 0, test MSE 0.328831
On step 100, test MSE 0.317843
On step 200, test MSE 0.109187
On step 300, test MSE 0.126465
On step 400, test MSE 0.150746
On step 500, test MSE 0.149150
On step 600, test MSE 0.132538
On step 700, test MSE 0.086550
On step 800, test MSE 0.045726
On step 900, test MSE 0.016989
Post training MSE: 0.0050751740077935505
Final weights:
(array([[-0.82126236, -1.0974077 , -0.72742885],
       [-0.5102594 ,  0.86615705,  0.56041175],
       [-0.58490086, -1.1651294 ,  0.21690042],
       [ 0.04502507, -1.1233195 ,  1.0100982 ],
       [-0.6560105 ,  0.01175537,  1.0444621 ],
       [-0.11602639, -0.27310133, -0.33843368],
       [ 0.36227196,  0.18799226, -1.1815559 ],
       [-0.5743607 ,  0.40260196,  0.92558235],
       [-0.5172754 ,  0.974886  , -0.20048371],
       [-0.84515256,  0.79922104, -0.85205144]], dtype=float32), array([[-0.39935228],
       [ 0.08716074],
       [ 0.46455976],
       [ 0.99843484]